In [1]:
import json
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
import nltk
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing,feature_selection
## for explainer
from lime import lime_text
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
import transformers

In [3]:
review=pd.read_csv(r'C:\Users\11638\Favorites\Code\Project\Review_project_sentiment_wUser.csv')

In [4]:
review.head(2)

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date,...,language,text_clean,read_ease,polarity,subjectivity,Adj_ratio,review_count,NumElite,average_stars,NumofWords
0,1073206,JdReKgETiiJEDmshrO4TLw,pyarmAnR-i-qookQamqRTA,V2GOReqPvr8qpCC7sWfoTw,1.0,17,1,0,Just to let this car company that people DO re...,2014-03-06 12:38:52,...,en,let car company people read yelp check review ...,85.49,-0.500000,1.000000,0.120000,9,0,3.00,56
1,6229216,zL4se_Ixdcl8kvTOHCS3rg,s16-BUo-orUsELvMu5ocKg,VH0Ib9S3E-dxbQdQC4rffg,1.0,15,6,4,Mistral was the worst dining experience I have...,2010-07-22 18:08:01,...,en,mistral worst dining experience ever life bad ...,79.19,0.029864,0.502499,0.225888,1777,10,3.82,767


In [5]:
dtf_train, dtf_test = model_selection.train_test_split(review, test_size=0.3)
## get target
y_train = dtf_train["useful_level"].values
y_test = dtf_test["useful_level"].values

In [6]:
corpus1 = dtf_train["text_clean"]

## create list of lists of unigrams
lst_corpus = []
for string in corpus1:
    lst_words = string.split() 
    lst_grams = [" ".join(lst_words[i:i+1]) for i in range(0, len(lst_words), 1)]
    lst_corpus.append(lst_grams)

In [7]:
nlp = gensim.models.word2vec.Word2Vec(lst_corpus, vector_size=500,  window=8, min_count=1, sg=1)# sg=1 skip gram/sg=0 SBOW

In [8]:
## tokenize text
tokenizer = kprocessing.text.Tokenizer(lower=True, split=' ', 
                     oov_token="NaN", 
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(lst_corpus)
dic_vocabulary = tokenizer.word_index
## create sequence
lst_text2seq= tokenizer.texts_to_sequences(lst_corpus)
## padding sequence
X_train = kprocessing.sequence.pad_sequences(lst_text2seq, 
                    maxlen=100, padding="post", truncating="post")

In [9]:
i = 1

## list of text: ["I like this", ...]
len_txt = len(dtf_train["text_clean"].iloc[i].split())
print("from: ", dtf_train["text_clean"].iloc[i], "| len:", len_txt)

## sequence of token ids: [[1, 2, 3], ...]
len_tokens = len(X_train[i])
print("to: ", X_train[i], "| len:", len(X_train[i]))

## vocabulary: 
print("check: ", dtf_train["text_clean"].iloc[i].split()[0], 
      " -- idx in vocabulary -->", 
      dic_vocabulary[dtf_train["text_clean"].iloc[i].split()[0]])

print("vocabulary: ", dict(list(dic_vocabulary.items())[0:5]), "... (padding element, 0)")

from:  point received exceptional exceptional service mr kelly security entire team long story short attended concert older person could wait outside cold line wrapped around building block end sight mr kelly understood helped u get venue early allowed u extra time needed get seat without pummeled swarm aggressive people would certainly stampede rush venue mr kelly even one staff member check u midway show pretty rare find venue thats inclusive senior disabled understands need extra time impressed warmth kindness entire team ensuring venue could enjoyed security unsung hero venue thank mr kelly | len: 91
to:  [  186   365  1821  1821    12  1498  3942  1139   420   832   114   650
   434  2382  3070  1288   196    25    96   243   418   145  1849    65
   416  1219   187  2503  1498  3942  2570   770    19     5  1155   487
  1022    19   320     2   223     5   423   166 30320 14289  2451    26
     9   886 25484  1380  1155  1498  3942    17     3    59   570   136
    19  9670   257

In [10]:
corpus2 = dtf_test["text_clean"]

## create list of n-grams
lst_corpus2 = []
for string in corpus2:
    lst_words = string.split()
    lst_grams = [" ".join(lst_words[i:i+1]) for i in range(0, 
                 len(lst_words), 1)]
    lst_corpus2.append(lst_grams)

In [11]:
lst_text2seq2 = tokenizer.texts_to_sequences(lst_corpus2)

## padding sequence
X_test = kprocessing.sequence.pad_sequences(lst_text2seq2, maxlen=100,
             padding="post", truncating="post")

In [12]:
## start the matrix (length of vocabulary x vector size) with all 0s
embeddings = np.zeros((len(dic_vocabulary)+1, 300))
for word,idx in dic_vocabulary.items():
    ## update the row with vector
    try:
        embeddings[idx] =  nlp[word]
    ## if word not in model then skip and the row stays all 0s
    except:
        pass

In [13]:
X_train.shape

(38630, 100)

In [14]:
X_test.shape

(16557, 100)

In [15]:
y_test = dtf_test["useful_level"]
y_train= dtf_train["useful_level"]

In [23]:
EmbeddingTrain = pd.DataFrame(X_train) #Train set of word2vec features only
EmbeddingTest = pd.DataFrame(X_test)   #Test set of word2vec features only
EmbeddingTest

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,114,96,114,145,1064,16,13,8090,21,865,...,0,0,0,0,0,0,0,0,0,0
1,28,2,710,11523,11523,12271,8,125,207,42184,...,0,0,0,0,0,0,0,0,0,0
2,4495,3029,578,2370,1379,1175,588,1062,132,107,...,0,0,0,0,0,0,0,0,0,0
3,590,173,5,801,962,2143,371,111,454,6,...,8016,7805,1237,733,3946,41,2697,147,152,12195
4,200,159,1481,7103,269,22445,1283,48430,226,3011,...,45,20,260,179,204,11612,8598,265,1247,8598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16552,457,64,12,95,559,976,54,14745,2363,495,...,0,0,0,0,0,0,0,0,0,0
16553,159,10860,1403,287,318,51,853,908,43,980,...,576,74,9,38,202,859,286,780,6,2968
16554,604,64,12,1727,1413,124,726,11335,464,340,...,0,0,0,0,0,0,0,0,0,0
16555,3721,54,1425,105,61,85,3761,4018,333,495,...,237,334,5533,426,1112,52,108,0,0,0


In [24]:
AllotherTrain = dtf_train[["useful_level", "read_ease", "polarity", "subjectivity", "Adj_ratio", 'review_count', 'NumElite', 'NumofWords']]
AllotherTest = dtf_test[["useful_level", "read_ease", "polarity", "subjectivity", "Adj_ratio", 'review_count', 'NumElite', 'NumofWords']]

In [25]:
Xt1 = AllotherTrain.loc[:, AllotherTrain.columns != 'useful_level']     #Train set of all other features
yt1 = AllotherTrain.loc[:, AllotherTrain.columns == 'useful_level'].values  #Train y
Xv1 = AllotherTest.loc[:, AllotherTest.columns != 'useful_level']       #Test set of all other features
yv1 = AllotherTest.loc[:, AllotherTest.columns == 'useful_level'].values    #Test y

In [26]:
Xt1

,read_ease,polarity,subjectivity,Adj_ratio,review_count,NumElite,NumofWords
37710,96.69,0.600000,0.900000,0.000000,14,0,17
43155,67.28,0.174405,0.572487,0.241758,1039,4,166
51175,70.13,0.058333,0.263889,0.203390,1,0,109
35392,-62.51,0.308604,0.482468,0.210526,1154,1,157
26191,44.42,0.500000,0.600000,0.125000,115,0,60
...,...,...,...,...,...,...,...
16642,90.77,0.520833,0.711111,0.238095,303,0,36
45173,67.28,0.167246,0.543137,0.183908,1562,4,166
28171,-60.48,0.205357,0.577489,0.235955,499,7,155
5848,77.67,0.190000,0.474661,0.261538,2512,7,133


## Word2Vec-only evaluation

In [15]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import tree
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

In [17]:
model1 = BernoulliNB()
model1 = model1.fit(X_train, y_train)
y_predict1 = model1.predict(X_test)
print(accuracy_score(y_predict1, y_test))

0.510599746330857


In [22]:
## pipeline
#model = pipeline.Pipeline([("vectorizer", vectorizer),  
                           #("classifier", classifier)])
## train classifier
model1.fit(X_train, y_train.astype(int))
## test
#X_test = dtf_test["text_clean"].values
predicted = model1.predict(X_test)
#predicted = np.argmax(predicted,axis=1)
predicted_prob = model1.predict_proba(X_test)

In [25]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
accuracy = metrics.accuracy_score(y_test, predicted)
auc = metrics.roc_auc_score(y_test, predicted_prob, 
                            multi_class="ovr")
print("Accuracy:",  round(accuracy,2))
print("Auc:", round(auc,2))
print("Detail:")
print(metrics.classification_report(y_test, predicted))

Accuracy: 0.51
Auc: 0.71
Detail:
              precision    recall  f1-score   support

           0       0.69      0.75      0.72      7234
           1       0.32      0.06      0.10      5231
           2       0.35      0.67      0.46      4092

    accuracy                           0.51     16557
   macro avg       0.45      0.49      0.43     16557
weighted avg       0.49      0.51      0.46     16557



In [18]:
f1 = f1_score( y_test, y_predict1, average='macro' )
p = precision_score(y_test, y_predict1, average='macro')
r = recall_score(y_test, y_predict1, average='macro')

print(f1, p, r)

0.42605600264947213 0.4543702220461592 0.491379336806338


In [28]:
model2 =  RandomForestClassifier(random_state=4)
model2 = model2.fit(X_train, y_train) 
y_predict2 = model2.predict(X_test)
print(accuracy_score(y_predict2, y_test))

0.5624207283928248


In [27]:
model3 =  GradientBoostingClassifier(n_estimators= 50, learning_rate = 0.1,random_state=4)
model3 = model3.fit(X_train, y_train)
y_predict3 = model3.predict(X_test)
print(accuracy_score(y_predict3, y_test))

0.5681584828169355


In [29]:
model4 = tree.DecisionTreeClassifier(random_state=4)
model4 = model4.fit(X_train, y_train) 
y_predict4 = model4.predict(X_test)
print(accuracy_score(y_predict4, y_test))

0.45817479011898293


In [30]:
model5 =  SGDClassifier(loss="squared_loss")
model5 = model5.fit(X_train, y_train)
y_predict5 = model5.predict(X_test)
print(accuracy_score(y_predict5, y_test))

0.30440297155281754


## Emsemble Methods

## Other features + Word2Vec single classifier

In [27]:
for i in ["useful_level", "read_ease", "polarity", "subjectivity", "Adj_ratio", 'review_count', 'NumElite', 'NumofWords']:
    EmbeddingTrain[i] = AllotherTrain[i].values
EmbeddingTrain.head(3)

,0,1,2,3,4,5,6,7,8,9,...,98,99,useful_level,read_ease,polarity,subjectivity,Adj_ratio,review_count,NumElite,NumofWords
0,541,730,1877,125,5254,3165,1559,552,129,118,...,0,0,0,96.69,0.600000,0.900000,0.000000,14,0,17
1,186,365,1821,1821,12,1498,3942,1139,420,832,...,0,0,1,67.28,0.174405,0.572487,0.241758,1039,4,166
2,402,202,610,68,1360,586,3877,127,247,126,...,0,0,0,70.13,0.058333,0.263889,0.203390,1,0,109


In [28]:
NewXTrain = EmbeddingTrain.loc[:, EmbeddingTrain.columns != 'useful_level']
NewYTrain = EmbeddingTrain.loc[:, EmbeddingTrain.columns == 'useful_level']

In [29]:
for i in ["useful_level", "read_ease", "polarity", "subjectivity", "Adj_ratio", 'review_count', 'NumElite', 'NumofWords']:
    EmbeddingTest[i] = AllotherTest[i].values
NewXTest = EmbeddingTest.loc[:, EmbeddingTest.columns != 'useful_level']
NewYTest = EmbeddingTest.loc[:, EmbeddingTest.columns == 'useful_level'].values

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn import svm
from sklearn.metrics import accuracy_score

In [35]:
logreg = LogisticRegression(max_iter=5000)
logreg.fit(NewXTrain, NewYTrain)
print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(logreg.score(NewXTrain, NewYTrain)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
     .format(logreg.score(NewXTest, NewYTest)))

C:\Users\11638\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Accuracy of Logistic regression classifier on training set: 0.60
Accuracy of Logistic regression classifier on test set: 0.60
